In [29]:
import pyspiel
import pickle
import numpy as np
import pandas as pd
from open_spiel.python.algorithms import cfr, outcome_sampling_mccfr, expected_game_score, exploitability, get_all_states_with_policy
from open_spiel.python.examples.ubc_mccfr_cpp_example import state_to_final
from open_spiel.python.algorithms import get_all_states
from tqdm import tqdm

In [30]:
params = dict()
params['filename'] = 'parking_with_instants_1.json'
game = pyspiel.load_game_as_turn_based('clock_auction', params)

In [31]:
information_state_string_to_state = get_all_states.get_all_states(game, depth_limit=-1, include_terminals=False, include_chance_states=False, include_mean_field_states=False, to_string=lambda s: s.information_state_string(), stop_if_encountered=False)

In [40]:
# root = '/shared/cfr/parking_with_instants/1/cfr_3_123'
root = '/shared/cfr/parking_with_instants/1/mccfr_0_0'

In [43]:
records = []
for t in tqdm(range(1, 10_000)):
#     with open(f'{root}/cfr_{t}.pkl', 'rb') as f:
    with open(f'{root}/mccfr_external_{t}.pkl', 'rb') as f:
        solver = pickle.load(f)
        info_state_values_table = solver.info_state_values_table()
        for info_state_string, state in information_state_string_to_state.items():
            player = state.current_player()
            info_state_tensor = state.information_state_tensor()
            for i, action in enumerate(state.legal_actions()):
#                 regret = info_state_values_table[info_state_string].cumulative_regrets[i]
                regret = info_state_values_table[info_state_string].instantaneous_regrets[i]
                record = dict(player=player, info_state_tensor=info_state_tensor, action=action, regret=regret, t=t)
                  # Commented out to save memory. Only useful for debugging
#                 record['info_state_string'] = info_state_string 
                records.append(record)
df = pd.DataFrame.from_records(records)

100%|██████████| 9999/9999 [03:51<00:00, 43.21it/s]  


In [44]:
df

,player,info_state_tensor,action,regret,t
0,0,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",0,27.272726,1
1,0,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",1,-2.727274,1
2,0,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",2,36.161615,1
3,1,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",0,-1.111111,1
4,1,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",1,0.000000,1
...,...,...,...,...,...
24947500,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...",2,0.000000,9999
24947501,0,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",0,0.000000,9999
24947502,1,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",0,-50.000000,9999
24947503,1,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...",1,0.000000,9999


In [45]:
df.to_csv("training_mccfr_0.csv", index=False)

In [ ]:
len(information_state_string_to_state)

In [ ]:
dir(solver)

In [ ]:
# # For an MCCFR solver, you need to do this because the implementation only calls this on copies and so when it serializes these nodes, the current policy actually isn't updated. 
# # See https://github.com/newmanne/open_spiel/blob/master/open_spiel/algorithms/external_sampling_mccfr.cc#L145
# # Possible I'm not understanding something and it's subtly important why this occurs in this way
# l = list(info_state_values_table.values())
# for i in range(len(l)):
#     l[i].apply_regret_matching()
# for i in range(5):
#     print(l[i].current_policy)